# LDA Topic Modelling

* This notebook is showcases the process of building an NLP Topic Model using `Latent Dirichlet Allocation` method. 
* The dataset we are going to use are `title` and `soft title` from `apify_dataset_clean.csv`. 

## Table Of Contents

## Installations


In [1]:
# ## installing required libraries
! pip install beautifulsoup4
! pip install pandas
! pip install numpy
! pip install plotly
! pip install nbformat
! pip install ipykernel
! pip install matplotlip
! pip install wordcloud
! pip install gensim
! pip install pyLDAvis
! pip install nltk
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_trf 


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Using cached setuptools-65.6.3-py3-none-any.whl (1.2 MB)
  Using cached wheel-0.38.4-py3-none-any.whl (36 kB)


ERROR: To modify pip, please run the following command:
D:\workspace\Aletheia\env\Scripts\python.exe -m pip install -U pip setuptools wheel

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 460.3/460.3 MB 5.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
from pprint import pprint

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

# loading library
import pickle

from joblib import dump, load

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\workspace\Aletheia\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\workspace\Aletheia\env\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


## Reading Data

In [3]:
## reading manaully scrapped data
data = pd.read_csv('../data/apify_dataset_clean.csv')
print(data.shape)

(11003, 13)


## Utility Functions

### Preparing Stop Words

In [33]:
## extending stopwords
# lets break down the cleaning functions into smaller functions
stop_words = nltk.corpus.stopwords.words('english')

## trying to remove stopwords from stopwords super set. 
stopwords_super_set = pd.read_csv("../data/stopwords/sw1k.csv")

## filtering stopwords to pronouns and other type
stopwords_to_remove = list(stopwords_super_set.loc[(stopwords_super_set["type"] == "G" ), "term"])


# stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'say', 'one', 'time', 'people',
#                   'know', 'like', 'tell', 'get', 'year', 'go', 'around', 'award', 'actually', 'carry',
#                    'new', 'it', 'show', 'news', 'go', 'fox', 'make', 'do', 'not', 'say',
#                    'also', 'love', 'it', 'star', 'go', 'do', 'say', 'not', 'said'
#                    ])

# stop_words.extend(stopwords_to_remove)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Common Utility Functions

In [5]:
nlp = spacy.load('en_core_web_trf')
# Utility Functions for Text Cleaning
def sent_to_words(sentences):
    for sentence in sentences:
        yield (simple_preprocess(str(sentence), deacc=True))

# function to clean html tags from text


def clean_html(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

# function to convert text to lowercase


def lower_case(text):
    return text.lower()

# function to remove line breaks


def remove_line_breaks(text):
    return re.sub(r'\n', '', text)

# function to remove punctuation


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# function to remove numbers


def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# function to remove extra spaces


def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

# function to remove stopwords


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# helper function to create pos tags


def create_pos_tag(str_sent):
    return nlp(str_sent)

# function for text lemmatization using spac
##'ADJ', 'VERB'
def lemmatization(texts, allowed_postags=['PROPN', 'NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if (token.pos_ in allowed_postags and token.is_alpha and token.is_stop == False)])
    return texts_out

def lemmatization_without_pos(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc])
    return texts_out


def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

## helper function to create pos tags distribution
def create_pos_tags_distribution(docs = []):
    token_distribution = {}
    is_alpha = 0
    is_stop = 0
    for doc in docs:
        for token in doc:
            token_distribution[token.pos_] = token_distribution.get(token.pos_, 0) + 1
            if(token.is_alpha):
                is_alpha += 1
            if(token.is_stop):
                is_stop += 1
    return token_distribution, is_alpha, is_stop


# function to create n-grams from noun chunks
def create_noun_chunk_ngrams(docs):
    n_gram_docs = []
    for doc in docs:
        doc_text = doc.text
        for chunk in doc.noun_chunks:
            chunk_n_gram = "_".join(chunk.text.split(" "))
            doc_text = doc_text.replace(chunk.text, chunk_n_gram)
        n_gram_docs.append(doc_text.split(" "))
    return n_gram_docs


def lemmatization_noun_chunks(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if (
            ("_" in token.text) or ## if the token is a noun chunk allow that
            (token.pos_ in ['NOUN', 'PROPN'] and token.is_alpha and token.is_stop == False) ## if the token is a noun or proper noun allow that
        )])
    return texts_out

### Gensim Models Utility Functions

In [6]:
## function to compute optimal parameters for LDA model
def compute_coherence_values(dictionary, corpus, id2word, texts, num_topics, passes, chunk_sizes=[200]):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    params = []
    for num_topic in num_topics:
        for chunk_size in chunk_sizes:
            for num_passes in passes:
                model = LdaModel(corpus=corpus,
                                 id2word=id2word,
                                 num_topics=num_topic,
                                 random_state=100,
                                 update_every=1,
                                 chunksize=chunk_size,
                                 passes=num_passes,
                                 per_word_topics=True)
                model_list.append(model)
                coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_lda = coherencemodel.get_coherence()
                coherence_values.append(coherence_lda)
                params.append({'num_topics': num_topic, 'chunk_size': chunk_size, 'passes': num_passes})

    return model_list, coherence_values, params

def analyze_gensim_lda_model(lda_model, corpus, id2word, texts, num_topics, passes, chunk_sizes=[200]):
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

## helper functions to visualize LDA model
def visualize_gensim_lda_model(lda_model, corpus, id2word, filename="gensim_lda.html"):
    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda_model, corpus, id2word)
    vis.save(filename)

### Sklearn Model Utility Functions

In [7]:
import numpy as np


# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)


def print_sklearn_sparcity(data_vectorized):
    # Materialize the sparse data
    data_dense = data_vectorized.todense()

    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


def create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized):
    lda_output = lda_model.transform(data_vectorized)
    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]
    
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    return df_document_topic

def print_sklearn_dominant_topics(lda_model, data_vectorized):
    df_document_topic = create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized)
    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    return df_document_topics

def print_sklearn_topic_distribution(lda_model, data_vectorized):
    df_document_topic = create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized)
    df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents").rename(columns={'index':'Topic'})
    # df_topic_distribution.columns = ["Topic Num", "Num Documents"]
    return df_topic_distribution


# Show top n keywords for each topic
def show_sklearn_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

def format_sklearn_topics(topic_keywords):
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    return df_topic_keywords

def analyze_sklearn_lda_model(lda_model, data_vectorized):
    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda_model.score(data_vectorized))
    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda_model.perplexity(data_vectorized))

## helper function to visualize lda model
def visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer, mds='tsne'):    
    pyLDAvis.enable_notebook()
    panel2 = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds=mds)
    return panel2

## EDA on Text Data

In [11]:
## quick and dirty test to see if we should use `text` or `description` column. Using the longer column assuming it has more data. 
data[data['text'].str.len() < data["description"].str.len()]

,url,author,date,title,soft_title,description,text,day,month,year,month_name,word_count,line_count


In [12]:
## check for duplicates
data["text"].duplicated().sum()

3241

In [13]:
## there are 3287 duplicates in text. But the URLs are different. But since we are using text for topic modelling, we will remove duplicates
data.drop_duplicates(subset ="text", inplace = True)

In [14]:
## sanity checking for duplicates
data["text"].duplicated().sum()

0

In [16]:
data['text_word_count'] = data['text'].apply(lambda x: len(str(x).split(" ")))

data['text_word_count'].describe()


count    7762.000000
mean      591.073692
std       396.921273
min       150.000000
25%       354.250000
50%       490.000000
75%       715.000000
max      7630.000000
Name: text_word_count, dtype: float64

##### Notes
* Couple of interesting outliers, a title with 23 words and 4 words. 
* Lets check them out. 

In [18]:
## checkout outliers 
data[data["text_word_count"] > 1000]["text"]

16       FIRST ON FOX: Republican Senate nominee Hersch...
29       A woman from Connecticut who gave birth on a f...
38       EXCLUSIVE: Transportation Secretary Pete Butti...
40       Pull my tongue down. Happy Monday. I hope you ...
46       At this point in time, it looks like Republica...
                               ...                        
10963    Another left-leaning media narrative with fact...
10967    Joe Berlinger is hopeful that, in his lifetime...
10972    An Idaho man is revisiting some of the most pa...
10973    Beverly Enright has always believed her sister...
10993    Shantara Hurry has chosen to relive the most h...
Name: text, Length: 797, dtype: object

In [19]:
## checking the distribution of word count in title
fig = px.histogram(data, x="text_word_count", title="Distribution of Word Count in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [24]:
data[data["text_word_count"] > data["text_word_count"].quantile(0.75)].shape[0]

1939

##### Notes
* I wonder if certain large articles would affect the vocab and topi modelling.  

In [25]:
## lets create POS tags for each title and see the distribution of POS tags
# docs = [nlp(text) for text in data['text']]
docs = data["text"].apply(create_pos_tag)

KeyboardInterrupt: 

In [16]:
## creating pos tags distribution
token_distribution, is_alpha, is_stop = create_pos_tags_distribution(docs)

In [17]:
## convert the dictionary to a dataframe
token_distribution_df = pd.DataFrame.from_dict(token_distribution, orient='index', columns=['count']).reset_index().rename(columns={"index": "tags"})


In [18]:
## lets create a distribution of POS tags
## checking the distribution of word count in title
fig = px.histogram(token_distribution_df, x="tags", y="count", title="Distribution POS Tags in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [19]:
## lets see how many words are alpha and how many are stop words
print(f"we have total {data['text_word_count'].sum()} words in the title. Out of which {is_alpha} are alpha and {is_stop} are stop words")

we have total 101176 words in the title. Out of which 100416 are alpha and 29908 are stop words


##### Notes
* So `maximum` tags are
    * `PROPN`- proper noun
    * `VERB` - verb
    * `ADP` - adposition
    * `NOUN` - noun
    * `PUNCT` - punctuation
    * `ADJ` - adjective
* Since these are news article titles, I think useful tags are, 
    * `PROPN`
    * `NOUN`
    * `VERB`
    * `ADJ` - Not sure about adjective yet. 
* We can remove rest of the words and still have a decent topic model. 
* We can also use the `is_stop` and `is_alpha` tags to remove the stopwords and non alpha tokens.
    * Lets update the helper functions accordingly. 

## Text Pre-processing

In [42]:
def preprocess_text(text):
     text = clean_html(text)
     text = lower_case(text)
     text = remove_line_breaks(text)
     text = remove_punctuation(text)
     text = remove_numbers(text)
     text = remove_extra_spaces(text)
     return text

data["cleaned_text"] = data["text"].apply(preprocess_text)

### Tokenizing

In [43]:
## Just taking a random sample to see if this works
sample_data = data.sample(1000)

In [44]:
data_words = list(sent_to_words(sample_data['cleaned_text']))

### Creating Bigram & Tigram Models

In [45]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[10]]])

['los_angeles_county', 'district_attorney', 'george_gascon', 'won', 'over', 'voters', 'last_year', 'with', 'promises', 'of', 'criminal_justice_reform', 'and', 'has', 'largely', 'stuck', 'to', 'his', 'commitments', 'introducing', 'sweeping', 'changes', 'that', 'have', 'been', 'praised', 'by', 'some', 'while', 'simultaneously', 'roiling', 'detractors', 'and', 'his', 'fellow', 'chief', 'taking', 'office', 'in', 'december', 'hes', 'has', 'issued', 'directives', 'to', 'halt', 'bail', 'requests', 'stop', 'trying', 'juveniles', 'as', 'adults', 'stop', 'seeking', 'the', 'death_penalty', 'and', 'banned', 'prosecutors', 'from', 'seeking', 'sentencing', 'enhancements', 'and', 'from', 'attending', 'some', 'parole', 'hearingsthe', 'move', 'against', 'enhancements', 'which', 'can', 'result', 'in', 'harsher', 'punishments', 'for', 'crimes', 'such', 'as', 'using', 'gun', 'has', 'been', 'particularly', 'controversialin', 'response', 'to', 'the', 'dramatic', 'reforms', 'in', 'the', 'nations', 'largest',

## Gensim LDA with BOW

### Lemmatization

In [23]:


# Form Bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'imminent', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [25]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('imminent', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [26]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=50,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [27]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(41,
  '0.135*"threat" + 0.120*"paul" + 0.085*"sexual" + 0.082*"assault" + '
  '0.074*"trial" + 0.055*"allegation" + 0.040*"hollywood" + 0.018*"danny" + '
  '0.016*"atlanta" + 0.011*"masterson"'),
 (34,
  '0.163*"coronavirus" + 0.123*"military" + 0.089*"good" + 0.072*"week" + '
  '0.050*"michigan" + 0.047*"leadership" + 0.036*"loan" + 0.028*"movie" + '
  '0.020*"auction" + 0.017*"beijing"'),
 (30,
  '0.121*"texas" + 0.107*"people" + 0.090*"twitter" + 0.063*"elon" + '
  '0.058*"musk" + 0.050*"austin" + 0.044*"jeff" + 0.043*"tom" + 0.040*"team" + '
  '0.038*"fire"'),
 (23,
  '0.231*"national" + 0.048*"tribute" + 0.038*"keith" + 0.030*"sean" + '
  '0.023*"loretta" + 0.022*"lynn" + 0.020*"urban" + 0.018*"dolly" + '
  '0.011*"parton" + 0.009*"lamborghini"'),
 (17,
  '0.148*"mom" + 0.100*"invasion" + 0.094*"troop" + 0.050*"agent" + '
  '0.047*"government" + 0.044*"answer" + 0.033*"bear" + 0.030*"early" + '
  '0.026*"chevrolet" + 0.018*"fame"'),
 (19,
  '0.138*"life" + 0.134*"campaign" + 0.1

### Analyzing Model

In [28]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -38.671961070092735

Coherence Score:  0.5753319359586817


### Visualize Topics

In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
26    -0.245316  0.338748       1        1  3.941030
25    -0.156214 -0.174988       2        1  3.621490
30    -0.138909 -0.109782       3        1  3.174167
21    -0.085236 -0.033094       4        1  2.962985
33    -0.113293 -0.060559       5        1  2.958050
7     -0.108087 -0.053663       6        1  2.875687
39    -0.079055 -0.028727       7        1  2.820528
40    -0.055674 -0.015839       8        1  2.783456
38    -0.049563 -0.013131       9        1  2.740921
2     -0.067575 -0.021809      10        1  2.721676
8     -0.034233 -0.007192      11        1  2.672450
47    -0.074036 -0.025547      12        1  2.627087
9     -0.047223 -0.012152      13        1  2.608798
42    -0.066642 -0.021292      14        1  2.606143
34    -0.022548 -0.003392      15        1  2.452923
44    -0.039534 -0.009240      16        1  2.438919
13    -0.055042 -0.015497      17        1  2.429920
16    -0.010244 -0.000328      18        1  2.098310
22     0.010791  0.004452      19        1  2.022976
32     0.002478  0.002623      20        1  1.966902
35     0.018178  0.005896      21        1  1.963579
29    -0.000237  0.002067      22        1  1.933858
36     0.013217  0.004848      23        1  1.922220
18     0.016257  0.005539      24        1  1.922175
45     0.035850  0.008504      25        1  1.920228
24     0.019766  0.006215      26        1  1.894165
5      0.010178  0.004325      27        1  1.863857
19     0.030007  0.007702      28        1  1.850657
1      0.034591  0.008293      29        1  1.802940
37     0.032012  0.007944      30        1  1.772371
41     0.037705  0.008768      31        1  1.768907
28     0.051618  0.011577      32        1  1.743851
11     0.017654  0.005515      33        1  1.733478
10     0.024271  0.006746      34        1  1.728614
0      0.033188  0.008174      35        1  1.720572
49     0.032028  0.007865      36        1  1.667774
20     0.044787  0.009559      37        1  1.630484
3      0.044336  0.009530      38        1  1.610432
17     0.056536  0.010853      39        1  1.482593
4      0.057022  0.010891      40        1  1.440418
43     0.064200  0.011361      41        1  1.411351
14     0.079320  0.012193      42        1  1.310862
48     0.081154  0.013164      43        1  1.247169
12     0.076551  0.012105      44        1  1.203377
31     0.076047  0.012088      45        1  1.191177
23     0.084799  0.012188      46        1  1.146947
46     0.092372  0.011198      47        1  0.816747
27     0.090582  0.008434      48        1  0.591593
15     0.090582  0.008434      49        1  0.591593
6      0.090582  0.008434      50        1  0.591593, topic_info=       Term         Freq        Total Category  logprob  loglift
1     biden  1119.000000  1119.000000  Default  30.0000  30.0000
340     new   775.000000   775.000000  Default  29.0000  29.0000
235   trump   536.000000   536.000000  Default  28.0000  28.0000
12    house   459.000000   459.000000  Default  27.0000  27.0000
802  report   471.000000   471.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
24     kari     0.036664    19.686181  Topic50  -9.1207  -1.1558
25    katie     0.036664    15.963405  Topic50  -9.1207  -0.9461
26     lake     0.036664    31.580326  Topic50  -9.1207  -1.6284
27      law     0.036664   154.093573  Topic50  -9.1207  -3.2134
28     race     0.036664   397.191377  Topic50  -9.1207  -4.1603

[2308 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
70       11  0.995646     abortion
426      28  0.907662      academy
662      35  0.930587   accusation
678       6  0.952199      accuser
371       5  0.974439          act
...     ...       ...          ...
184      37  0.994474         year
902      15  0.861240        years
108      42  0.944822  yellowston

## Gensim LDA with Bigram BOW

### Lemmatization

In [30]:
## lemmatization with bigrams
data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'imminent', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [31]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [32]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('imminent', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [33]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [34]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(3,
  '0.146*"official" + 0.109*"border" + 0.083*"policy" + 0.075*"student" + '
  '0.062*"political" + 0.055*"arizona" + 0.029*"view" + 0.025*"release" + '
  '0.024*"government" + 0.022*"answer"'),
 (22,
  '0.174*"trump" + 0.091*"house" + 0.087*"race" + 0.068*"russian" + '
  '0.064*"people" + 0.063*"american" + 0.051*"twitter" + 0.040*"history" + '
  '0.035*"senate" + 0.032*"drug"'),
 (20,
  '0.095*"bidens" + 0.082*"liberal" + 0.076*"white" + 0.065*"hunter" + '
  '0.042*"speech" + 0.039*"joe" + 0.033*"business" + 0.027*"hospital" + '
  '0.023*"event" + 0.021*"call"'),
 (23,
  '0.201*"ukraine" + 0.128*"crime" + 0.065*"deal" + 0.058*"dems" + '
  '0.057*"mayor" + 0.037*"red" + 0.036*"tie" + 0.023*"possible" + '
  '0.022*"award" + 0.017*"decade"'),
 (21,
  '0.139*"election" + 0.137*"man" + 0.096*"coronavirus" + 0.075*"record" + '
  '0.069*"california" + 0.069*"america" + 0.035*"baby" + 0.032*"number" + '
  '0.019*"seat" + 0.015*"tribute"'),
 (0,
  '0.245*"police" + 0.169*"democrats" + 0.0

##### Notes
* So visually it seems we have a different topics when we use `bigrams`. 

### Analyzing Model

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -27.769318340007636

Coherence Score:  0.5775860625357506


### Visualize Topics

In [36]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
22    -0.385785  0.070737       1        1  6.104296
17    -0.053738 -0.353149       2        1  4.817247
25    -0.024131 -0.054377       3        1  4.229670
7     -0.006660 -0.015919       4        1  4.004654
21    -0.007476 -0.017243       5        1  3.929859
6     -0.010533 -0.022490       6        1  3.853419
2      0.001965 -0.003971       7        1  3.776684
18     0.010686  0.005463       8        1  3.754118
9     -0.000827 -0.007310       9        1  3.752397
0      0.007082  0.001743      10        1  3.650409
23     0.005011 -0.000471      11        1  3.578416
10    -0.099188  0.050510      12        1  3.568998
3      0.014647  0.008789      13        1  3.485920
8      0.025591  0.016846      14        1  3.444790
20     0.019266  0.012676      15        1  3.435280
13     0.021297  0.013909      16        1  3.322663
29     0.022757  0.015000      17        1  3.175640
19     0.028233  0.018350      18        1  3.011867
24     0.026639  0.017355      19        1  2.904567
11     0.032791  0.020781      20        1  2.844974
5      0.029457  0.018896      21        1  2.760093
16     0.035720  0.021920      22        1  2.729354
1      0.032874  0.020614      23        1  2.706052
28     0.033967  0.021017      24        1  2.675146
15     0.037241  0.022494      25        1  2.671243
12     0.036873  0.022294      26        1  2.567805
4      0.038337  0.022986      27        1  2.542577
26     0.041414  0.023977      28        1  2.343096
14     0.041034  0.023802      29        1  2.265132
27     0.045456  0.024768      30        1  2.093630, topic_info=            Term         Freq        Total Category  logprob  loglift
1          biden  1063.000000  1063.000000  Default  30.0000  30.0000
301          new   544.000000   544.000000  Default  29.0000  29.0000
11           gop   515.000000   515.000000  Default  28.0000  28.0000
735       report   484.000000   484.000000  Default  27.0000  27.0000
452       police   437.000000   437.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
7     california     0.065428   134.173618  Topic30  -9.6535  -3.7597
9    competitive     0.065428    11.230610  Topic30  -9.6535  -1.2792
10      election     0.065428   267.108306  Topic30  -9.6535  -4.4482
11           gop     0.065428   515.875602  Topic30  -9.6535  -5.1064
12         house     0.065428   271.270651  Topic30  -9.6535  -4.4637

[1197 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
62       16  0.994721     abortion
381       9  0.910684      academy
599       2  0.930007   accusation
615      14  0.965793      accuser
331       2  0.977091          act
...     ...       ...          ...
36       11  0.931098        wrong
6        30  0.934934           xi
163      23  0.994186         year
96       13  0.944202  yellowstone
478      30  0.689525          zoo

[530 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 18, 26, 8, 22, 7, 3, 19, 10, 1, 24, 11, 4, 9, 21, 14, 30, 20, 25, 12, 6, 17, 2, 29, 16, 13, 5, 27, 15, 28])

## Gensim LDA with Trigram BOW

### Lemmatization

In [37]:
## lemmatization with trigrams
data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])


[['biden', 'xi', 'meeting', 'imminent', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [38]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [39]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('imminent', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [40]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [41]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(3,
  '0.118*"car" + 0.068*"green" + 0.038*"november" + 0.037*"event" + '
  '0.028*"emotional" + 0.026*"jennifer" + 0.025*"newsom" + 0.025*"tennessee" + '
  '0.019*"pete" + 0.016*"dark"'),
 (22,
  '0.145*"gop" + 0.070*"election" + 0.058*"race" + 0.044*"military" + '
  '0.040*"story" + 0.035*"campaign" + 0.034*"democratic" + 0.033*"law" + '
  '0.032*"gov" + 0.032*"senate"'),
 (20,
  '0.211*"day" + 0.130*"time" + 0.083*"leader" + 0.079*"national" + '
  '0.033*"program" + 0.027*"park" + 0.023*"illinois" + 0.018*"ben" + '
  '0.017*"famous" + 0.012*"press"'),
 (23,
  '0.201*"man" + 0.128*"family" + 0.088*"good" + 0.067*"inflation" + '
  '0.027*"night" + 0.025*"comment" + 0.018*"founder" + 0.010*"boyfriend" + '
  '0.008*"cher" + 0.008*"chef"'),
 (21,
  '0.106*"prison" + 0.078*"company" + 0.031*"demand" + 0.018*"mandatory" + '
  '0.013*"mississippi" + 0.007*"shift" + 0.000*"conviction" + '
  '0.000*"therapist" + 0.000*"canada" + 0.000*"extortion"'),
 (0,
  '0.161*"home" + 0.095*"violence" + 

### Analyzing Model

In [42]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -27.97983575510768

Coherence Score:  0.5600343755347889


### Visualize Topics

In [43]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
22     0.309005 -0.218833       1        1  7.100145
28     0.231971  0.334217       2        1  6.738930
4      0.063033  0.036523       3        1  4.843114
26     0.122645 -0.072220       4        1  4.813064
10     0.077264 -0.060602       5        1  4.216545
18     0.011509  0.015071       6        1  3.920305
9      0.003680  0.012454       7        1  3.630344
17    -0.016244  0.006419       8        1  3.492285
24    -0.017321  0.006068       9        1  3.459093
7     -0.020007  0.005297      10        1  3.364273
12    -0.020382  0.005124      11        1  3.336112
5     -0.023232  0.004424      12        1  3.274075
27     0.001020 -0.025826      13        1  3.204258
20    -0.026507  0.003435      14        1  3.198344
6     -0.030227  0.002488      15        1  3.068371
25    -0.035507  0.001144      16        1  3.066391
14    -0.030169  0.002496      17        1  3.001295
13    -0.036656  0.000901      18        1  2.962009
8     -0.006978 -0.028117      19        1  2.899503
11    -0.040233 -0.000072      20        1  2.886466
23    -0.040528 -0.000192      21        1  2.788711
16    -0.040178 -0.000059      22        1  2.770924
19    -0.043531 -0.000937      23        1  2.661528
15    -0.049399 -0.002373      24        1  2.626210
2     -0.046241 -0.001610      25        1  2.607958
29    -0.052489 -0.003152      26        1  2.493326
1     -0.055634 -0.003983      27        1  2.289108
3     -0.058821 -0.004783      28        1  2.026151
0     -0.064032 -0.006335      29        1  1.771626
21    -0.065811 -0.006968      30        1  1.489534, topic_info=            Term         Freq        Total Category  logprob  loglift
1          biden  1061.000000  1061.000000  Default  30.0000  30.0000
204        trump   546.000000   546.000000  Default  29.0000  29.0000
301          new   543.000000   543.000000  Default  28.0000  28.0000
734       report   496.000000   496.000000  Default  27.0000  27.0000
450       police   437.000000   437.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
6             xi     0.059047    19.438821  Topic30  -9.3995  -1.5900
7     california     0.059047   136.712278  Topic30  -9.3995  -3.5406
9    competitive     0.059047    11.184648  Topic30  -9.3995  -1.0373
10      election     0.059047   240.591348  Topic30  -9.3995  -4.1058
11           gop     0.059047   495.926024  Topic30  -9.3995  -4.8291

[1206 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
62        2  0.994085     abortion
379      25  0.871053      academy
598      22  0.925704   accusation
614       8  0.966842      accuser
331      19  0.961182          act
...     ...       ...          ...
36        3  0.923172        wrong
6        19  0.925982           xi
163       7  0.993902         year
96        8  0.953975  yellowstone
476       9  0.729979          zoo

[526 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 29, 5, 27, 11, 19, 10, 18, 25, 8, 13, 6, 28, 21, 7, 26, 15, 14, 9, 12, 24, 17, 20, 16, 3, 30, 2, 4, 1, 22])

## Gensim LDA with Spacy noun-chunks n-Grams

##### Notes
* In this case we are trying to use `Spacy's` noun-chunks to create n-grams.
* We'll need to first create tokens from clean text. 
* We'll then need a function to replace the nouns in (noun chunks) with ngram word. 
* The `lemmatize` this, 
    * We'll need to test with and without POS to see if our filtering affects ngrams. 

### Creating Noun Chunks

In [44]:
## Lets start by reviewing noun chunks created by spacy
docs = [create_pos_tag(" ".join(x)) for x in data_words]

### Creating Noun Chunks Ngrams

In [45]:
n_grams = create_noun_chunk_ngrams(docs)
n_grams

[['biden',
  'says',
  'after',
  'xi',
  'meeting',
  'he',
  'doesn',
  'believe',
  'there',
  'is',
  'imminent_attempt',
  'from',
  'china',
  'to',
  'invade',
  'taiwan'],
 ['gop_rep_calvert',
  'wins',
  'election',
  'in',
  'competitive_california_house_seat'],
 ['pelosi',
  'not',
  'even',
  'thinking',
  'about',
  'political_future',
  'before',
  'house_majority',
  'called',
  'spox',
  'says'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'defeats',
  'gop_challenger',
  'kari_lake',
  'race',
  'may',
  'now',
  'go',
  'to',
  'recount',
  'under',
  'state_law'],
 ['idaho',
  'quadruple',
  'student',
  'homicide',
  'crime',
  'of',
  'passion',
  'burglary',
  'gone',
  'wrong',
  'among',
  'possible_motives',
  'mayor',
  'says'],
 ['jennifer_siebel_newsom',
  'identifies',
  'harvey_weinstein',
  'as',
  'alleged_rapist',
  'in',
  'emotional_sexual_assault_testimony'],
 ['commanders',
  'hand',
  'eagles',
  'first_loss',
  'of',
  'season',
  'in',

In [46]:
data_lemmatized = lemmatization_without_pos(n_grams)

In [47]:
data_lemmatized

[['biden',
  'say',
  'after',
  'xi',
  'meeting',
  'he',
  'doesn',
  'believe',
  'there',
  'be',
  'imminent_attempt',
  'from',
  'china',
  'to',
  'invade',
  'taiwan'],
 ['gop_rep_calvert',
  'win',
  'election',
  'in',
  'competitive_california_house_seat'],
 ['pelosi',
  'not',
  'even',
  'think',
  'about',
  'political_future',
  'before',
  'house_majority',
  'call',
  'spox',
  'say'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'defeat',
  'gop_challenger',
  'kari_lake',
  'race',
  'may',
  'now',
  'go',
  'to',
  'recount',
  'under',
  'state_law'],
 ['idaho',
  'quadruple',
  'student',
  'homicide',
  'crime',
  'of',
  'passion',
  'burglary',
  'go',
  'wrong',
  'among',
  'possible_motive',
  'mayor',
  'say'],
 ['jennifer_siebel_newsom',
  'identify',
  'harvey_weinstein',
  'as',
  'alleged_rapist',
  'in',
  'emotional_sexual_assault_testimony'],
 ['commander',
  'hand',
  'eagle',
  'first_loss',
  'of',
  'season',
  'in',
  'wild_nfc_east

##### Notes
* So since we are doing `lemmatization` without any `POS` some parts of speech words are present in our lemmatized data. 
* Words like `doesn't`, `believe`, `think` etc are present. I think we should update the function to ignore certain `POS` rather than just include all words. 
* Lets do a quick check on how these `POS` are distributed. 

In [48]:
n_gram_pos = [create_pos_tag(" ".join(x)) for x in data_lemmatized]
n_gram_pos_distribution, is_alpha, is_stop = create_pos_tags_distribution(n_gram_pos)

In [49]:
token_distribution_df = pd.DataFrame.from_dict(n_gram_pos_distribution, orient='index', columns=['count']).reset_index().rename(columns={"index": "tags"})
## lets create a distribution of POS tags
## checking the distribution of word count in title
fig = px.histogram(token_distribution_df, x="tags", y="count", title="Distribution POS Tags in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



##### Notes
* Lets see how our `ngrams` are tagged

In [50]:
for token in n_gram_pos:
    for token in token:
        ## Only print the noun chunks
        if("_" in token.text):            
            print(token.text, token.pos_, token.dep_)

imminent_attempt NOUN attr
gop_rep_calvert PROPN nsubj
competitive_california_house_seat PROPN pobj
political_future NOUN pobj
house_majority NOUN dep
katie_hobbs PROPN appos
gop_challenger NOUN dobj
kari_lake PROPN conj
state_law NOUN pobj
possible_motive ADJ amod
jennifer_siebel_newsom PROPN nsubj
harvey_weinstein PROPN dobj
alleged_rapist NOUN pobj
emotional_sexual_assault_testimony NOUN pobj
first_loss NOUN appos
wild_nfc_east_upset NOUN pobj
arizona_gop_rep_schweikert PROPN dep
democratic_challenger PROPN dobj
house_majority PROPN pobj
marjorie_taylor_greene PROPN dep
gop_colleague PROPN pobj
speaker_bad_strategy PROPN pobj
student_loan_buyout_abortion NOUN nsubj
distasteful_rape_comment ADJ dobj
special_warriors_broadcast PROPN pobj
oregons_th_congressional_district ADJ amod
democrat_andrea_salinas PROPN pobj
andy_warhol_art NOUN pobj
kevin_costner PROPN nsubj
yellowstone_national_park PROPN dobj
patriot_awards PROPN npadvmod
honor_heroism NOUN dobj
fox_nation PROPN pobj
this_day

##### Notes
* Looks like most of the noun chunks are tagged as `NOUN`, `PROPN` or `ADJ`, but lets confirm it using visualization. 


In [51]:
noun_chunks_tags = {}
for token in n_gram_pos:
    for token in token:
        ## Only print the noun chunks
        if("_" in token.text):
            ## increment the count of the noun chunk
            noun_chunks_tags[token.pos_] = noun_chunks_tags.get(token.pos_, 0) + 1

noun_chunks_tags

{'NOUN': 8112,
 'PROPN': 8452,
 'ADJ': 714,
 'X': 851,
 'VERB': 74,
 'NUM': 30,
 'ADV': 10,
 'PRON': 56,
 'INTJ': 4,
 'SYM': 4,
 'ADP': 1,
 'DET': 1}

##### Notes
* So as assumed most of them are `NOUN` or `PROPN` but just to make sure we don't loose any chunks, lets modify our lemmatization script to handle this case. 

In [52]:
data_lemmatized = lemmatization_noun_chunks(n_grams)

In [53]:
data_lemmatized

[['biden', 'xi', 'meeting', 'imminent_attempt', 'china', 'taiwan'],
 ['gop_rep_calvert', 'election', 'competitive_california_house_seat'],
 ['pelosi', 'political_future', 'house_majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop_challenger',
  'kari_lake',
  'race',
  'recount',
  'state_law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'possible_motive',
  'mayor'],
 ['jennifer_siebel_newsom',
  'harvey_weinstein',
  'alleged_rapist',
  'emotional_sexual_assault_testimony'],
 ['commander', 'eagle', 'first_loss', 'season', 'wild_nfc_east_upset'],
 ['arizona_gop_rep_schweikert',
  'democratic_challenger',
  'republicans',
  'house_majority'],
 ['marjorie_taylor_greene',
  'gop_colleague',
  'mccarthy',
  'speaker_bad_strategy'],
 ['student_loan_buyout_abortion', 'midterm', 'democrats', 'biden', 'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'distasteful_rape_comment',
  'special_warriors_broadcast'],
 ['oregons_th

##### Notes
* So the lemmatized data looks promising lets train the model and see. 

### Create Dictionary & Corpus

In [54]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [55]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('biden', 1),
  ('china', 1),
  ('imminent_attempt', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [56]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [57]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(18,
  '0.079*"america" + 0.020*"fox_nation" + 0.019*"baby" + 0.007*"flight" + '
  '0.006*"kevin_costner" + 0.000*"massive_shift" + 0.000*"nightmare_scenario" '
  '+ 0.000*"mccaul" + 0.000*"nato_majority" + 0.000*"red_line"'),
 (17,
  '0.156*"year" + 0.045*"threat" + 0.027*"life" + 0.010*"kathie_lee_gifford" + '
  '0.007*"cher" + 0.004*"atlanta" + 0.000*"democracy" + 0.000*"massive_shift" '
  '+ 0.000*"possible_threat" + 0.000*"nato_majority"'),
 (20,
  '0.009*"joe_biden" + 0.008*"loss" + 0.006*"chief" + 0.005*"the_nation" + '
  '0.000*"mccaul" + 0.000*"nato_majority" + 0.000*"massive_shift" + '
  '0.000*"quantum_computing_answer" + 0.000*"nuclear_weapons" + '
  '0.000*"possible_threat"'),
 (23,
  '0.027*"veteran" + 0.016*"christmas" + 0.015*"thanksgiving" + '
  '0.014*"tribute" + 0.009*"jeff" + 0.008*"team" + 0.003*"tom_brady" + '
  '0.002*"saturday" + 0.000*"massive_shift" + 0.000*"mccaul"'),
 (10,
  '0.085*"iran" + 0.000*"massive_shift" + 0.000*"red_line" + '
  '0.000*"nuclear_weap

### Analyzing Model

In [58]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -48.68804192741965

Coherence Score:  0.5222654255319291


##### Notes
* So our model didn't perform well, in fact regular model BOW did better than noun chunks. 

### Visualize Topics

In [59]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.146446 -0.124518       1        1  5.383394
22    -0.147752 -0.123232       2        1  5.279413
12     0.000172  0.033807       3        1  3.997223
7      0.000138  0.026461       4        1  3.959367
17     0.000145  0.028044       5        1  3.818324
28     0.000095  0.017702       6        1  3.599469
9      0.000097  0.018143       7        1  3.562110
11     0.000105  0.019678       8        1  3.539652
1      0.000078  0.014193       9        1  3.475027
8      0.000071  0.012979      10        1  3.420717
4      0.000088  0.016261      11        1  3.398471
14     0.000090  0.016677      12        1  3.372617
15     0.000051  0.009251      13        1  3.322350
29     0.000060  0.011018      14        1  3.238948
18     0.000039  0.007153      15        1  3.200399
26     0.000052  0.009520      16        1  3.173025
6      0.000044  0.008080      17        1  3.145606
21     0.000027  0.005040      18        1  3.135344
2      0.000022  0.004076      19        1  3.014306
23     0.000015  0.002799      20        1  2.999242
24     0.000011  0.002080      21        1  2.973060
0      0.000010  0.001826      22        1  2.970302
5      0.000005  0.001183      23        1  2.941265
27     0.000005  0.000999      24        1  2.915346
10     0.000010  0.002002      25        1  2.810244
25    -0.000007 -0.000969      26        1  2.782167
20    -0.000025 -0.004309      27        1  2.697947
19    -0.000019 -0.003147      28        1  2.697408
3     -0.000032 -0.005407      29        1  2.644502
16    -0.000043 -0.007390      30        1  2.532754, topic_info=            Term        Freq       Total Category  logprob  loglift
0          biden  566.000000  566.000000  Default  30.0000  30.0000
552        trump  282.000000  282.000000  Default  29.0000  29.0000
85       ukraine  183.000000  183.000000  Default  28.0000  28.0000
113         year  185.000000  185.000000  Default  27.0000  27.0000
46     democrats  149.000000  149.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
16     kari_lake    0.044116   10.784575  Topic30  -9.7827  -1.8232
17   katie_hobbs    0.044116    3.921392  Topic30  -9.7827  -0.8115
18          race    0.044116   48.846987  Topic30  -9.7827  -3.3338
22         crime    0.044116   61.232429  Topic30  -9.7827  -3.5597
28       student    0.044116   39.296899  Topic30  -9.7827  -3.1162

[1242 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
432      29  0.820751             accusation
131      15  0.982652                america
213       2  0.916608                 answer
13        8  0.956293                arizona
553      11  0.776839                   army
376       5  0.843243                atlanta
58        8  0.927979                auction
530      13  0.911639                 author
287      19  0.518465              authority
117      15  0.908312                   baby
695      13  0.693719  baby_formula_shortage
259      11  0.954207                 battle
0         1  0.998337                  biden
483      21  0.929532                 bidens
660      22  0.893289            bill_murray
693       7  0.934424                  birth
669      17  0.858994                brother
410      28  0.928670             california
381      13  0.937720                   call
502       9  0.918733                    car
708      22  0.728086                 chance
390       5  0.843451                   cher
440      27  0.754134                  chief
1        17  0.986006                  china
515      20  0.912598              christmas
33       21  0.788694              commander
305      26  0.530893  commanders_dan_snyder
452      21  0.921201                  court
22       14  0.979873                  crime
45        4 

## Sklearn LDA with Count Vectorization

##### Notes
* For Count Vecotorization we'l use SKLearn's LDA algorithm. The algorithm is same as Gensim, but the interface is different and it allows us to use CountVectorizations. 

### Lemmatization

In [46]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words)
print(data_lemmatized[:1])

KeyboardInterrupt: 

### Count Vectorizer

In [61]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [62]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.4122935089024177 %


### Building Topic Model

In [63]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [64]:
analyze_sklearn_lda_model(lda_model, data_vectorized)
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -271773.1160605511
Perplexity:  1401.0324474026772
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [65]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.670000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.710000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25
Doc2,0.170000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,5
Doc3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.080000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20
Doc4,0.000000,0.000000,0.000000,0.000000,0.000000,0.720000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5
Doc5,0.000000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,32
Doc6,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,24
Doc7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12
Doc9,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4


In [66]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,5,754
1,4,708
2,2,698
3,9,517
4,20,498
5,15,430
6,24,383
7,0,362
8,13,311
9,16,278


### Visualizing Topics

In [67]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
20     -44.368641   80.138664       1        1  6.526355
5     -138.421722  -16.030073       2        1  6.357104
24     139.511307   44.093506       3        1  5.724831
4     -101.665459  105.023949       4        1  5.391911
9     -127.418510  -79.483330       5        1  5.014847
15     166.477966  -15.980798       6        1  4.861046
25    -133.877090   48.780884       7        1  4.737908
2      -70.092300 -132.844467       8        1  4.555227
16     111.591011  -23.124155       9        1  4.546001
30     125.231026  102.090065      10        1  4.366233
22      85.746155   20.327995      11        1  4.215204
13      48.509293 -159.584808      12        1  4.023843
27     -61.989368  139.762741      13        1  3.999061
23      72.875450   68.735046      14        1  3.306240
32     137.058807  -80.033066      15        1  3.265377
11      -8.285363 -138.945923      16        1  2.871504
12      21.853781   81.509132      17        1  2.822626
14     -79.703957   37.631767      18        1  2.781858
17       0.113067  132.265686      19        1  2.574167
0       73.370575  -58.725014      20        1  2.184850
19      87.966118 -113.855835      21        1  2.100883
26     -70.151825  -72.523422      22        1  2.076643
10      62.565025  127.370605      23        1  2.001352
7      -80.441696  -18.947187      24        1  1.319103
28     -18.144257  -84.579926      25        1  1.223406
6      -30.916740  -37.252106      26        1  1.181266
21      33.541790  -92.791672      27        1  1.064533
29      -8.704215   44.170010      28        1  1.015524
3      -38.988415   10.102205      29        1  0.950074
18      17.785284  -47.666271      30        1  0.836380
8        3.805821   -4.596910      31        1  0.801479
31      34.836906   26.998297      32        1  0.664831
1       48.804962  -15.165370      33        1  0.638332, topic_info=              Term       Freq       Total Category  logprob  loglift
94           biden  598.00000  598.000000  Default  30.0000  30.0000
724            new  401.00000  401.000000  Default  29.0000  29.0000
487          house  274.00000  274.000000  Default  28.0000  28.0000
911         russia  175.00000  175.000000  Default  27.0000  27.0000
876         report  244.00000  244.000000  Default  26.0000  26.0000
...            ...        ...         ...      ...      ...      ...
38       americans    0.06774   54.462983  Topic33  -8.1704  -1.6355
269            dem    0.06774   74.083100  Topic33  -8.1704  -1.9432
871   relationship    0.06774   21.921426  Topic33  -8.1704  -0.7255
153       ceremony    0.06774   15.980455  Topic33  -8.1704  -0.4094
1126           war    0.06774  123.398241  Topic33  -8.1704  -2.4534

[1280 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         9  0.937859     abbott
2         6  0.275257   abortion
2         9  0.718062   abortion
3         1  0.985805     abrams
4        15  0.961644      abuse
...     ...       ...        ...
1166     22  0.981592      young
1167      6  0.950335   youngkin
1168     15  0.951166     zeldin
1169      4  0.937958  zelenskyy
1170     21  0.931025     zucker

[915 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 6, 25, 5, 10, 16, 26, 3, 17, 31, 23, 14, 28, 24, 33, 12, 13, 15, 18, 1, 20, 27, 11, 8, 29, 7, 22, 30, 4, 19, 9, 32, 2])

In [68]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,military,national,veteran,study,high,michigan,cnns,service,james,americas,missile,tapper,jet,future,debt,racial,whitmer,datum,account,personal
Topic 1,nicolle,wallace,skin,watchdog,samantha,toll,winner,riot,gavin,service,loss,sexual,pentagon,desperate,jim,limit,human,ray,surge,big
Topic 2,cnn,death,china,group,book,taiwan,chris,month,nuclear,scandal,queen,green,money,chinese,final,mother,prince,mark,dark,source
Topic 3,game,joy,oklahoma,legend,tribute,mlb,reid,legacy,crt,field,ray,irans,skin,democrats,french,republicans,family,jet,service,school
Topic 4,democrats,biden,war,russian,president,world,climate,washington,protest,change,post,victim,critic,activist,message,lawmaker,effort,loan,cop,obama
Topic 5,trump,crime,people,family,expert,pelosi,iran,parent,twitter,dem,murder,elon,musk,mayor,suspect,social,congress,college,political,shooting
Topic 6,russia,health,brother,trip,afghan,mental,talk,peace,brittney,skin,update,dolly,citizen,pandemic,twin,effect,illinois,backtoschool,day,president
Topic 7,afghanistan,security,fan,withdrawal,anniversary,johnny,aoc,pentagon,instagram,ground,depp,guest,global,amber,appeal,defamation,wallace,nicolle,nicole,disney
Topic 8,officer,fentanyl,nypd,vehicle,area,bail,watchdog,year,operation,ukrainian,tiktok,chinas,abortion,protest,journalist,south,supreme,mlb,police,sheriff
Topic 9,white,day,star,student,america,history,home,liberal,dead,dems,son,black,illegal,member,music,west,troop,tie,elvis,immigrant


##### Notes
* Somehow I couldn't find out `coherence` for but from visualization it seems that the model has created topics which are easy to interpret and not overlapping
* I still see some topics that don't make sense but, we can tweak this further by creating n-grams, including more POS and skipping lemmatization. 

### Write to Pickle File

In [69]:
# create an iterator object with write permission - model.pkl
features = vectorizer.get_feature_names_out()

path = "../pickles/sklearn_count_vectorization/"

with open(path + '/features_v1', 'wb') as files:
    pickle.dump(features, files)
    
with open(path + '/model_v1', 'wb') as files:
    pickle.dump(lda_model, files)   

## Sklearn LDA with Bi-Grams Count Vectorization

### Lemmatization with Bigrams

In [70]:
## lemmatization with bigrams
data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
data_lemmatized = lemmatization(data_words_bigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'imminent', 'attempt', 'china', 'taiwan']]


In [71]:
data_lemmatized

[['biden', 'xi', 'meeting', 'imminent', 'attempt', 'china', 'taiwan'],
 ['gop',
  'rep',
  'calvert',
  'election',
  'competitive',
  'california',
  'house',
  'seat'],
 ['pelosi', 'political', 'future', 'house', 'majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'gop',
  'challenger',
  'race',
  'recount',
  'state',
  'law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'wrong',
  'possible',
  'motive',
  'mayor'],
 ['jennifer', 'siebel', 'newsom', 'rapist', 'emotional', 'testimony'],
 ['commander', 'eagles', 'loss', 'season', 'wild', 'nfc', 'east', 'upset'],
 ['arizona',
  'gop',
  'rep',
  'schweikert',
  'democratic',
  'challenger',
  'republicans',
  'house',
  'majority'],
 ['marjorie',
  'taylor',
  'greene',
  'gop',
  'colleague',
  'mccarthy',
  'speaker',
  'bad',
  'strategy'],
 ['buyout', 'abortion', 'midterm', 'democrats', 'biden', 'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'distasteful',
  'rape',
  'comment',


### Count Vectorizer

In [72]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [73]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.39075112722766375 %


### Building Topic Model

In [74]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [75]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -216628.62659080783
Perplexity:  1449.4553342887064
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [76]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.170000,0.010000,0.010000,27
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.490000,0.000000,0.000000,0.150000,0.000000,0.230000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.010000,0.010000,0.010000,0.010000,14
Doc3,0.000000,0.000000,0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.000000,19
Doc4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580000,0.150000,0.150000,0.000000,29
Doc5,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.760000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.810000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,22
Doc7,0.000000,0.000000,0.000000,0.110000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.000000,14
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.670000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,23
Doc9,0.010000,0.510000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,1


In [77]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,1,888
1,0,592
2,3,590
3,14,532
4,4,532
5,2,519
6,5,466
7,27,333
8,11,312
9,8,301


In [78]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
14      -8.440456   90.149948       1        1  6.284583
27    -137.038803  -52.116375       2        1  5.673662
29     146.141403  169.368118       3        1  5.524074
1     -136.767929   28.056456       4        1  5.468919
19     196.323929 -125.734802       5        1  4.647795
4       58.820442  140.294312       6        1  4.555965
2       59.228130   62.402313       7        1  4.146597
3      -79.468018   76.570572       8        1  4.126733
23     110.345886 -128.916473       9        1  3.961788
28     210.854919   59.768673      10        1  3.821620
11     -91.822350 -229.005783      11        1  3.782273
5     -125.597343 -142.703461      12        1  3.551582
31    -123.440041  155.322418      13        1  3.286328
24     -44.415028 -158.762253      14        1  3.274865
16    -216.386703   -6.412930      15        1  3.265090
8      208.688339  -31.097879      16        1  3.257783
32    -204.522064 -107.480278      17        1  3.242887
18      11.242757 -234.749268      18        1  2.951610
9       39.909458 -156.380966      19        1  2.890396
0      128.710709  -57.965359      20        1  2.479196
20      22.445618  214.086868      21        1  2.469576
22     124.893288   11.836060      22        1  2.397793
13     111.213120 -211.575562      23        1  2.105356
26     130.216415   86.183571      24        1  2.087274
10    -186.032623   91.658569      25        1  1.952104
30     -41.507648  157.507614      26        1  1.934169
17     -70.172028   -2.621508      27        1  1.525525
7      -68.811028  -77.715607      28        1  1.240251
25      -2.757716  -99.584312      29        1  1.022638
21      53.263062   -5.383107      30        1  0.856850
12      55.783127  -68.777847      31        1  0.832579
15      -8.929082  -36.549698      32        1  0.738762
6       -6.708246   22.086742      33        1  0.643377, topic_info=          Term        Freq       Total Category  logprob  loglift
80       biden  564.000000  564.000000  Default  30.0000  30.0000
736     report  260.000000  260.000000  Default  29.0000  29.0000
599        new  267.000000  267.000000  Default  28.0000  28.0000
216        day  221.000000  221.000000  Default  27.0000  27.0000
901      trump  250.000000  250.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
321      fight    0.118656   20.568145  Topic33  -7.3863  -0.1091
802     silent    0.118656   15.020468  Topic33  -7.3863   0.2053
683  president    0.118656   96.124530  Topic33  -7.3863  -1.6510
746   rhetoric    0.118656   12.089710  Topic33  -7.3863   0.4223
474        kid    0.118656   89.475529  Topic33  -7.3863  -1.5793

[1236 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0        11  0.931169          abc
1        14  0.992460     abortion
2        19  0.966655        abuse
4        18  0.933546       access
5        17  0.944626      account
...     ...       ...          ...
976      15  0.952317  yellowstone
977       8  0.954146        young
978      18  0.919145     youngkin
979       5  0.893193       zeldin
980       4  0.938388    zelenskyy

[861 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 28, 30, 2, 20, 5, 3, 4, 24, 29, 12, 6, 32, 25, 17, 9, 33, 19, 10, 1, 21, 23, 14, 27, 11, 31, 18, 8, 26, 22, 13, 16, 7])

In [79]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,military,veteran,life,congress,michigan,john,study,teacher,cnns,fox,americas,nick,troop,james,tom,datum,honor,announcement,personal,ready
Topic 1,democrats,midterm,voter,russian,president,gop,leader,video,campaign,afghanistan,critic,violence,drug,battle,power,owner,program,career,use,obama
Topic 2,senate,race,candidate,state,georgia,final,washington,mark,major,nevada,control,role,prosecutor,center,close,warnock,secretary,real,line,battleground
Topic 3,year,republicans,covid,medium,case,yearold,party,news,taliban,young,girl,old,nominee,list,mcconnell,night,hearing,williams,long,date
Topic 4,biden,bidens,record,hunter,energy,district,speech,chris,joe,oil,administration,business,cop,head,immigration,congressional,evidence,domestic,title,foreign
Topic 5,star,child,family,parent,murder,country,support,way,suspect,photo,christmas,doc,father,service,point,celebrity,illegal,relationship,doctor,set
Topic 6,levin,defamation,love,relationship,woke,victim,nick,congressional,prolife,idea,hard,russian,fake,emotional,parent,local,facility,navy,end,upcoming
Topic 7,free,summer,lee,mike,pentagon,god,jan,faith,jennifer,apology,emotional,independent,newsom,riot,winner,vulnerable,wedding,capitol,focus,utah
Topic 8,border,crisis,migrant,policy,virginia,shooting,response,federal,probe,university,ban,assault,test,official,harris,weapon,football,player,cuomo,fight
Topic 9,key,primary,health,trial,tiktok,court,viral,allegation,massive,husband,secret,rule,abuse,community,lawyer,pound,bid,couple,marriage,spike


### Write to Pickle File

In [80]:
# create an iterator object with write permission - model.pkl
features = vectorizer.get_feature_names_out()

path = "../pickles/sklearn_count_vectorization_bigrams/"

with open(path + '/features_v1', 'wb') as files:
    pickle.dump(features, files)
    
with open(path + '/model_v1', 'wb') as files:
    pickle.dump(lda_model, files)   

## SKLearn LDA with noun-chunks n-grams

### Creating Noun Chunks

In [81]:
## Lets start by reviewing noun chunks created by spacy
docs = [create_pos_tag(" ".join(x)) for x in data_words]

### Creating Noun Chunks N-grams

In [82]:
n_grams = create_noun_chunk_ngrams(docs)

In [83]:
## lemmatization with bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
# data_lemmatized = lemmatization_noun_chunks(n_grams)
data_lemmatized = lemmatization(n_grams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'china', 'taiwan']]


In [84]:
data_lemmatized

[['biden', 'xi', 'meeting', 'china', 'taiwan'],
 ['election'],
 ['pelosi', 'spox'],
 ['arizona', 'gov', 'election', 'race', 'recount'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'wrong',
  'mayor'],
 [],
 ['commander', 'eagle', 'season'],
 ['republicans'],
 ['mccarthy'],
 ['midterm', 'democrats', 'biden', 'critic'],
 ['hall', 'famer', 'tim', 'hardaway'],
 [],
 ['mercedes', 'auction'],
 ['november'],
 ['history', 'love'],
 ['walker', 'start'],
 [],
 ['record'],
 ['republicans', 'willing'],
 ['ukraine', 'taiwan', 'xi'],
 ['door', 'official'],
 ['disappointed'],
 ['decade'],
 ['biden', 'summit', 'indonesia', 'concern'],
 [],
 ['tribute', 'nt'],
 ['worth', 'year'],
 [],
 ['baby', 'flight'],
 [],
 ['arizona'],
 [],
 ['america', 'violence'],
 ['paint', 'traffic', 'unhelpful'],
 ['twitter', 'people', 'austin'],
 ['mississippi', 'demand'],
 ['arizona', 'rancher', 'ms'],
 [],
 ['broadway', 'lawsuit', 'white'],
 ['greg', 'gutfeld'],
 [],
 ['republicans', 'demo

### Count Vectorizer

In [85]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [86]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.32756994386503624 %


### Building Topic Model

In [87]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [88]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -34671.21402862915
Perplexity:  285.73554488924265
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [89]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.260000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,1
Doc1,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.520000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,15
Doc2,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.520000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,25
Doc3,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.260000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,15
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.760000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,17
Doc5,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0
Doc6,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0
Doc7,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.520000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,12
Doc8,0.520000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0
Doc9,0.010000,0.210000,0.010000,0.010000,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,5


In [90]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,0,3835
1,1,486
2,31,204
3,4,163
4,5,157
5,11,145
6,12,144
7,14,138
8,29,137
9,21,137


In [91]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
1     -128.943893  -65.541389       1        1  6.719991
31     120.114670  -71.778297       2        1  5.386487
29    -135.123810    6.740100       3        1  4.083119
12     -22.106522  125.171402       4        1  3.655083
21     137.287216    0.169824       5        1  3.558482
30    -105.847099   68.653862       6        1  3.288795
5      -73.778252  -78.064629       7        1  3.229397
11      93.252983   22.534081       8        1  3.176407
23     -85.815063   22.386641       9        1  3.136175
14     -44.290096 -131.640152      10        1  3.125417
24     -59.063766   86.226700      11        1  3.105301
20     101.667397   70.824814      12        1  3.096447
10     -25.800537  -89.231972      13        1  3.077845
22     -10.545510  -52.490055      14        1  3.018195
32     -13.571925   74.619278      15        1  2.961496
4       68.853592  -99.611809      16        1  2.915963
13     -87.228889  -25.058809      17        1  2.839012
19      28.146875  106.921951      18        1  2.817575
27      24.098869  -73.321655      19        1  2.774061
16      17.985403   50.655277      20        1  2.769420
7       17.350475 -117.883141      21        1  2.743906
17      52.363285   30.178976      22        1  2.689634
6       91.549400  -25.590900      23        1  2.677848
0      -47.005405  -39.380836      24        1  2.600101
8       55.443027   72.416107      25        1  2.555791
15      58.058327  -51.213570      26        1  2.512005
28     -13.627728   27.575529      27        1  2.436724
18      52.908298   -8.831724      28        1  2.392685
25     -47.210403   42.484215      29        1  2.340763
9      -46.528347    1.137592      30        1  2.202939
2       20.543976  -28.852673      31        1  2.095289
3      -12.403473  -12.187575      32        1  2.019271
26      17.981123    8.902758      33        1  1.998378, topic_info=          Term        Freq       Total Category  logprob  loglift
15       biden  259.000000  259.000000  Default    30.00  30.0000
62   democrats  123.000000  123.000000  Default    29.00  29.0000
239       year  109.000000  109.000000  Default    28.00  28.0000
214      trump  128.000000  128.000000  Default    27.00  27.0000
218    ukraine   99.000000   99.000000  Default    26.00  26.0000
..         ...         ...         ...      ...      ...      ...
95    governor    0.039098   16.639771  Topic33    -8.05  -2.1407
234        way    0.039098   21.579986  Topic33    -8.05  -2.4006
8       arrest    0.039098   15.319833  Topic33    -8.05  -2.0580
123   majority    0.039098   22.349639  Topic33    -8.05  -2.4357
165    protest    0.039098   18.956700  Topic33    -8.05  -2.2710

[1707 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0        19  0.984071       abortion
1        11  0.980935    afghanistan
2        31  0.926760          alarm
3        18  0.984914        america
4        10  0.977957      americans
5        18  0.889866         answer
6        28  0.956924            aoc
7         8  0.932342        arizona
8        30  0.913848         arrest
9        14  0.964151         attack
10       20  0.919432        auction
11       15  0.919019           baby
12        1  0.934423       backlash
13        2  0.944294            bad
14        8  0.941540         battle
15        1  0.997043          biden
16        3  0.901134         bidens
17       30  0.911594            big
18       25  0.939191          birth
19       11  0.948663           book
20        9  0.951084         border
21       21  0.927673     california
22       33  0.965064       campaign
23       13  0.897383            car
24        4  0.945568        carlson
25       24  0.955004           case
26       12  0.897566            cdc
27        1  0.921524         charge
28        8  0.902809 

In [92]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,inflation,reporter,case,pennsylvania,tie,kyiv,mccarthy,racist,wife,midterm,race,question,shooting,majority,nyc,fbi,dog,cop,police,halloween
Topic 1,biden,congress,war,taiwan,president,charge,backlash,crisis,deal,china,msnbc,home,fetterman,republican,update,europe,ready,zelenskyy,alarm,america
Topic 2,thousand,alarm,source,doctor,investigation,republican,hour,coronavirus,ready,fbi,death,china,war,taiwan,silent,video,fentanyl,expert,year,christmas
Topic 3,poll,debate,open,issue,taylor,big,god,midterm,crime,police,majority,warnock,gop,yearold,hour,official,dad,time,senate,alarm
Topic 4,year,judge,silent,pound,lot,lawsuit,group,republicans,ready,alarm,issue,life,border,war,woman,china,hour,florida,cher,zelenskyy
Topic 5,midterm,parent,critic,office,july,warnock,walker,democrats,biden,silent,trump,decade,case,economy,november,inflation,mccarthy,russia,democracy,plan
Topic 6,fetterman,texas,family,photo,dad,fentanyl,michigan,email,man,car,abortion,threat,ukraine,today,georgia,book,important,child,protester,case
Topic 7,day,covid,california,concern,cop,protester,help,biden,georgia,washington,election,important,poll,news,backlash,fentanyl,fox,love,safe,trump
Topic 8,woman,florida,taliban,birth,afghanistan,republicans,aoc,trump,question,iran,life,dems,mccarthy,year,right,bidens,gop,state,house,bad
Topic 9,man,governor,arrest,christmas,cuomo,thanksgiving,big,cher,obama,death,abortion,backlash,letter,suspect,fbi,birth,source,race,europe,people


### Write to Pickle File

In [93]:
# create an iterator object with write permission - model.pkl
features = vectorizer.get_feature_names_out()

path = "../pickles/sklearn_count_vectorization_noun_chunks"

with open(path + '/features_v1', 'wb') as files:
    pickle.dump(features, files)
    
with open(path + '/model_v1', 'wb') as files:
    pickle.dump(lda_model, files)    

## SKLearn with Tri-Grams

In [94]:
## lemmatization with bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
data_lemmatized = lemmatization_noun_chunks(data_words_trigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


In [95]:
data_lemmatized

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan'],
 ['gop', 'rep', 'calvert', 'election', 'california', 'house', 'seat'],
 ['pelosi', 'future', 'house', 'majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop',
  'challenger',
  'kari_lake',
  'race',
  'recount',
  'state',
  'law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'motive',
  'mayor'],
 ['jennifer',
  'siebel',
  'newsom',
  'harvey_weinstein',
  'rapist',
  'sexual_assault',
  'testimony'],
 ['commander', 'eagles', 'loss', 'season', 'wild', 'nfc', 'east', 'upset'],
 ['arizona',
  'gop',
  'rep',
  'schweikert',
  'challenger',
  'republicans',
  'house',
  'majority'],
 ['marjorie',
  'taylor',
  'greene',
  'gop',
  'colleague',
  'mccarthy',
  'speaker',
  'bad',
  'strategy'],
 ['student_loan',
  'buyout',
  'abortion',
  'midterm',
  'democrats',
  'biden',
  'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'rape',
  'comment',
  'special',
  

### Count Vectorizer

In [96]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [97]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.4063710841443506 %


### Building Topic Model

In [98]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [99]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -239448.82377936013
Perplexity:  1170.1017032659263
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [100]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.670000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,6
Doc1,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.720000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc2,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,1
Doc3,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.650000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.210000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,32
Doc6,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
Doc7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.620000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.670000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,17
Doc9,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.130000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19


In [101]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,1,733
1,0,719
2,2,642
3,10,515
4,3,423
5,6,407
6,18,378
7,7,370
8,4,310
9,5,297


In [102]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
10    -48.919239  -6.560654       1        1  5.290027
1     -76.166542  -9.485081       2        1  5.143586
18    -42.161819 -30.160267       3        1  5.119954
19    -61.560291  20.161993       4        1  4.395120
7     -20.547855 -73.373222       5        1  4.287313
6      42.657387  54.159672       6        1  4.276563
2     -59.494904  46.049797       7        1  4.231072
31     63.092125  28.903294       8        1  3.993138
22    -32.848267  38.553291       9        1  3.847977
0      47.102688 -55.161568      10        1  3.765510
3      41.606979 -20.812984      11        1  3.682147
24    -34.928986 -52.961620      12        1  3.631491
25    -27.789194  68.657478      13        1  3.311973
16     -9.278261  54.291599      14        1  3.276927
14     -4.074765 -50.408905      15        1  3.256721
21     65.272270 -27.642876      16        1  3.236050
4     -33.235867  14.516580      17        1  3.141900
26     60.697521   0.734041      18        1  3.028033
30     13.447790  37.045918      19        1  3.013138
23    -65.420425 -42.243137      20        1  2.809026
27     16.625397 -67.385681      21        1  2.787877
5      15.111651  65.908096      22        1  2.787014
9      34.946251  26.932556      23        1  2.508664
17     24.585428 -39.293068      24        1  2.317379
8     -17.276979 -28.724840      25        1  2.017943
32     34.247829   4.949896      26        1  1.736686
11      5.365294 -28.092175      27        1  1.695333
28     -8.912347  29.023096      28        1  1.691115
15    -24.295820  -7.494877      29        1  1.495035
13     18.897181 -10.370337      30        1  1.338506
12     11.329117  12.065025      31        1  1.101292
29     -1.645489  -8.162411      32        1  0.932863
20     -9.515964   8.701823      33        1  0.852626, topic_info=         Term        Freq       Total Category  logprob  loglift
90      biden  603.000000  603.000000  Default  30.0000  30.0000
439     house  280.000000  280.000000  Default  29.0000  29.0000
793    report  234.000000  234.000000  Default  28.0000  28.0000
987     trump  256.000000  256.000000  Default  27.0000  27.0000
294  election  241.000000  241.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
286     drone    0.050971   32.780174  Topic33  -8.6429  -1.7017
440     human    0.050971   31.511563  Topic33  -8.6429  -1.6623
393     guard    0.050971   13.425189  Topic33  -8.6429  -0.8090
883    source    0.050971   27.927942  Topic33  -8.6429  -1.5415
521      lake    0.050971   20.779465  Topic33  -8.6429  -1.2459

[1167 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0        12  0.967941     abbott
1        25  0.943479        abc
2         1  0.295540   abortion
2         3  0.703384   abortion
3         3  0.972414     abrams
...     ...       ...        ...
1070     17  0.946731   youngkin
1071     12  0.039435     zeldin
1071     21  0.907012     zeldin
1072      7  0.947387  zelenskyy
1073     15  0.955474     zucker

[950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 2, 19, 20, 8, 7, 3, 32, 23, 1, 4, 25, 26, 17, 15, 22, 5, 27, 31, 24, 28, 6, 10, 18, 9, 33, 12, 29, 16, 14, 13, 30, 21])

In [103]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,police,man,coronavirus,city,case,inflation,americans,study,christmas,team,austin,chicago,number,pandemic,arrest,msnbcs,loss,comment,dozen,girlfriend
Topic 1,border,official,migrant,climate,crisis,pelosi,expert,dem,policy,officer,protest,change,activist,lawmaker,mother,meghan,prince,cancer,illegal,southern
Topic 2,ukraine,war,president,florida,world,deal,veteran,month,nation,message,invasion,battle,national,effort,service,putin,air,americas,soldier,agent
Topic 3,election,midterm,time,bidens,record,money,liberal,mark,group,husband,dark,network,jim,clinton,evidence,cash,prolife,aid,manchin,politic
Topic 4,white,video,campaign,virginia,member,queen,response,king,owner,jan,charles,probe,university,ban,criticism,committee,weapon,sister,heart,performance
Topic 5,report,star,way,concern,oregon,putin,netflix,doc,chinas,regime,center,role,american,hearing,celebrity,summit,williams,doctor,fear,coach
Topic 6,biden,china,fbi,taiwan,admin,cop,source,meeting,kanye,point,win,commander,wave,years,reduction,backlash,matter,red,documentary,dan
Topic 7,house,gop,rep,california,dems,mayor,majority,primary,tim,ohio,ryan,seat,season,handout,kelly,letter,suicide,governor,challenger,work
Topic 8,speech,education,voting,job,drug,department,use,visit,missile,supporter,island,nbc,alaska,rise,reform,cbs,greg,choice,fauci,cartel
Topic 9,woman,child,nyc,support,board,nick,taliban,human,father,relationship,kim,attempt,model,kardashian,cannon,silence,kabul,secret,hate,flag


### Write to Pickle File

In [104]:
# create an iterator object with write permission - model.pkl
features = vectorizer.get_feature_names_out()

path = "../pickles/sklearn_count_vectorization_trigrams"

with open(path + '/features_v1', 'wb') as files:
    pickle.dump(features, files)
    
with open(path + '/model_v1', 'wb') as files:
    pickle.dump(lda_model, files)   

## Random Testing

##### Notes
* Lets just do a quick test with the models that just saved to `pickle files` 
* In order to do that we need to take the texts through following steps, 
    * `Tokenize`
    * `Create Noun Chunks`
    * `Create Noun Chunks n-grams`
    * `Lemmatization`
    * `Count Vectorization`
    * `LDA Transform`

### Utiliy Functions

In [105]:
# helper function to load the model and features
def load_pickle_files(path):
    with open(path + '/model_v1', 'rb') as model:
        lda = pickle.load(model)

    # assuming you pickled the vectorizer
    with open(path + '/features_v1', 'rb') as vocab:
        features = pickle.load(vocab)
    return (lda, features)


def create_noun_chunks(data_words):
    docs = [create_pos_tag(" ".join(x)) for x in data_words]
    return docs

In [106]:
# with open('../pickles/noun_chunks_model_v1', 'rb') as model:
#     lda = pickle.load(model)

# # assuming you pickled the vectorizer
# with open('../pickles/noun_chunks_features_v1', 'rb') as vocab:
#     features = pickle.load(vocab)

In [107]:
def predict_topics(text, model, features):
    # tokenize the text
    # print("tokenizing the text...")
    data_words = list(sent_to_words(text))

    # create noun chunks
    # print("creating noun chunks...")
    docs = create_noun_chunks(data_words)

    # create noun chunks ngrams
    # print("creating noun chunk ngrams...")
    n_grams = create_noun_chunk_ngrams(docs)

    # lemmatization
    # print("lemmatization noun chunks...")
    data_lemmatized = lemmatization_noun_chunks(n_grams)

    # count vectorization
    # print("count vectorization...")
   
    vectorizer = CountVectorizer(vocabulary=features)
    # column names
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]
    # print(topicnames)
    
    data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

    ## Create a dataframe with topics as rows and features as columns
    ## each cell represents the weight of the feature in the topic
    df_topic_keywords = pd.DataFrame(model.components_)
    df_topic_keywords.columns = vectorizer.get_feature_names_out()
    df_topic_keywords.index = topicnames  # type: ignore
    
    topic_keywords = show_sklearn_topics(vectorizer, model)
    ## transform gives us the topic distribution for each document
    ## here we have a list of probabilities for each topic, index of the list is the topic number
    topic_probability_scores = model.transform(data_vectorized)
    
    ## from the dataframe we then select the row with the highest probability
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()  # type: ignore
    topic_list = list(topic_keywords)[np.argmax(topic_probability_scores)]
    topic.sort(reverse=True)    
    features_with_weights = list(zip(topic_list, topic))
    return features_with_weights

### Test Data

In [108]:
random_text = data.sample(1)["cleaned_text"].tolist()
random_text

['dems view heavy biden districts as critical to fending off red wave as gop invests in blue regions report']

## Random Testing SKLearn Count Vectorization

### Load Model

In [109]:
(lda_count_vectorization, features_count_vectorization) = load_pickle_files("../pickles/sklearn_count_vectorization")

In [110]:

# import time
# for text in random_text:
#     print(text)
#     features_with_weights = predict_topics(text, lda_count_vectorization, features_count_vectorization)
#     print(features_with_weights)
#     print(" ")

features_with_weights = predict_topics(random_text, lda_count_vectorization, features_count_vectorization)
print(random_text)
print(features_with_weights)

['dems view heavy biden districts as critical to fending off red wave as gop invests in blue regions report']
[('trump', 280.55448538183555), ('crime', 132.6163347163977), ('people', 108.84953212722914), ('family', 101.75391929387855), ('expert', 94.6580956202156), ('pelosi', 91.0019549066333), ('iran', 90.09507535122398), ('parent', 86.05807486631515), ('twitter', 83.91695638204418), ('dem', 82.5762399240975), ('murder', 76.37489059926214), ('elon', 64.81557565946767), ('musk', 59.74809438303596), ('mayor', 58.870911199784544), ('suspect', 57.06770718439559), ('social', 56.94221575749434), ('congress', 54.31476541838948), ('college', 47.94782915300147), ('political', 46.749647692066105), ('shooting', 46.35388392316569)]


## Random Testing Count Vectorization With BiGrams

### Load Model

In [111]:
(lda_with_bigrams, features_with_bigrams) = load_pickle_files("../pickles/sklearn_count_vectorization_bigrams")

In [112]:
features_with_weights = predict_topics(random_text, lda_with_bigrams, features_with_bigrams)
print(random_text)
print(features_with_weights)

['dems view heavy biden districts as critical to fending off red wave as gop invests in blue regions report']
[('election', 184.47586595415905), ('gop', 151.1773165379656), ('gov', 84.53700290514163), ('world', 81.17048850321409), ('law', 73.128419975157), ('group', 67.17891170217602), ('gun', 66.23144218327981), ('right', 57.89213751893744), ('charge', 56.07743815702098), ('state', 53.957385797502766), ('msnbc', 45.583182879982225), ('message', 41.6536303396783), ('force', 35.486194901708615), ('doj', 35.31755789803698), ('kelly', 27.277097430517383), ('soldier', 25.262023979346736), ('nation', 25.139488232233465), ('leadership', 25.043781984240116), ('money', 24.589316843347834), ('suicide', 21.188924120437022)]


## Random Testing Count Vectorization With Tri-Grams

### Load Model

In [113]:
(lda_with_trigrams, features_with_trigrams) = load_pickle_files("../pickles/sklearn_count_vectorization_trigrams")

In [114]:
features_with_weights = predict_topics(random_text, lda_with_trigrams, features_with_trigrams)
print(random_text)
print(features_with_weights)

['dems view heavy biden districts as critical to fending off red wave as gop invests in blue regions report']
[('border', 167.94642292634515), ('official', 120.212625112185), ('migrant', 110.44505026157655), ('climate', 96.2437442327697), ('crisis', 90.02974597878959), ('pelosi', 89.04669155594534), ('expert', 87.9777801869514), ('dem', 79.63586751660893), ('policy', 77.65584119570926), ('officer', 56.83666428240679), ('protest', 56.39817495527367), ('change', 56.25619567840445), ('activist', 44.68958982974065), ('lawmaker', 41.261717617286486), ('mother', 36.32374855653028), ('meghan', 34.6300753386232), ('prince', 34.23058660329835), ('cancer', 31.60987389828605), ('illegal', 31.60493455005761), ('southern', 29.573740183989976)]


## Random Testing Count Vectorization With Noun Chunks

### Load Model

In [115]:
(lda_with_noun_chunks, features_with_noun_chunks) = load_pickle_files("../pickles/sklearn_count_vectorization_noun_chunks")

In [116]:
features_with_weights = predict_topics(random_text, lda_with_noun_chunks, features_with_noun_chunks)
print(random_text)
print(features_with_weights)

['dems view heavy biden districts as critical to fending off red wave as gop invests in blue regions report']
[('gop', 69.18908027244983), ('putin', 47.19427839105045), ('test', 14.51873081528801), ('shanghai', 10.995412760725362), ('drive', 10.48974494432873), ('number', 9.733335230690772), ('issue', 0.030303281606385537), ('dem', 0.030303122662068506), ('poll', 0.030303119095846005), ('war', 0.030303109085831823), ('crime', 0.030303108386869705), ('law', 0.03030310413787299), ('bad', 0.030303103355555302), ('georgia', 0.030303101569598607), ('msnbc', 0.030303101390165347), ('covid', 0.030303101271758014), ('book', 0.030303100807781404), ('question', 0.030303100298943026), ('response', 0.03030310008452407), ('dead', 0.03030309978227368)]
